<a href="https://colab.research.google.com/github/sezinbiner/find_path_genetic_algorithms/blob/main/v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
def try_routes(coming_routes, start, size):
  fitness_list = []
  return_routes = []
  route_matrix_list = []
  end=[0,0]
  
  for route in coming_routes:  #population 5000
    flag=0
    cost = 0            
    
    matrix = np.zeros((size,size))
    matrix[start[0], start[1]] = 1
    end[0]=start[0]
    end[1]=start[1]
    for i in range (len(route)):       #length of one route 9*9-1
      if (i !=0):
        add_move =  moves[route[i]-1]
        end = [end[0] + add_move[0], end[1] + add_move[1]]
        if (end[0] <= 8) and (end[1] <= 8) and (end[0] >= 0) and (end[1] >= 0): #kaç kişi siliniyor bak az silinen varsa create route
          cost += find_cost(route[i-1], route[i])
          matrix[end[0], end[1]] = 1
        else:  
          end = [end[0] - add_move[0], end[1] - add_move[1]]
          route[i]=find_way(end)
          add_move =moves[route[i]-1]
          end = [end[0] + add_move[0], end[1] + add_move[1]]
          cost += find_cost(route[i-1], route[i])
          matrix[end[0], end[1]] = 1  
    route_matrix_list.append(matrix)
    area = len(np.argwhere(matrix == 1))
    return_routes.append(route)
    
    fitness_list.append(find_fitness(start, end, cost, area))
  return return_routes,fitness_list,route_matrix_list

def group(routes,numberOfDrone,fitness_list, matrix_list) :
    i=0
    fitness=[]
    routes1_list=[]
    routes2_list=[]

    while i<(len(routes)):
        sum=similarity(matrix_list[i],matrix_list[i+1])
        fitness.append(find_fitness2(fitness_list[i], fitness_list[i+1], sum))
        routes1_list.append(routes[i])
        routes2_list.append(routes[i+1])
        i=i+2
    zipped = list(zip(routes1_list, routes2_list,fitness))   
      
    sorted_routes = sorted(zipped, key = lambda x: x[2])
    unzipped_list = list(zip(*sorted_routes))
    unzipped_list=list(unzipped_list)
    return unzipped_list
def create_first_route(start,numberOfDrone,length):
  fitness_list = []
  routes = []
  matrix_list = []
  
  for i in range (population*numberOfDrone):
    matrix = np.zeros((size,size))
    matrix[start[0], start[1]] = 1;
    end = copy.deepcopy(start)
    route = []
    cost = 0
    route.append(1)
    while len(route) <= length:
      route.append(find_way(end))
      cost += find_cost(route[-2], route[-1])
      add_move = moves[route[-1]-1]
      end[0] += add_move[0]
      end[1] += add_move[1]
      matrix[end[0], end[1]] = 1
    
    area = len(np.argwhere(matrix == 1))
    routes.append(route)
    matrix_list.append(matrix)
    fitness_list.append(find_fitness(start, end, cost, area))
    
  return routes, fitness_list,matrix_list


def calculate_distance(start,end):
    dist = math.sqrt(((start[0]-end[0])**2)+((start[1]-end[1])**2))
    return dist

def find_fitness(start, end, cost, area):
  distance = calculate_distance(start, end)
  fitness = (area) - cost - (distance)
  return fitness

def cross_over(routes,length):
  new_generation = []
  length=(length/2)+1
  count=0
  for i in range (len(routes)):

    route1 = random.choice(routes)
    route2 = random.choice(routes)
    #new_route1 = route1[:1]+route1[1:21] + route2[21:41] + route1[41:61] + route2[61:]
    #new_route2 = route2[:1]+route2[1:21] + route1[21:41] + route2[41:61] + route1[61:]
    new_route1 = route1[:1]+route1[1:int(length)] + route2[int(length):] 
    new_route2 = route2[:1]+route2[1:int(length)] + route1[int(length):]      

    new_generation.append(new_route1)
    new_generation.append(new_route2)

  return new_generation

def mutation(routes, mutation_ratio, length, population):

  for i in range(int(mutation_ratio)):
    index = random.randint(0,population-1)
    route = routes[index]        #route sayısı çok azaldığı için hata alıyoruz
    for i in range (3):
      index2 = random.randint(1,length)
      direction = random.randint(1,8)
      route[index2] = direction
    routes[index] = route
  return routes



def similarity(mat1,mat2):
  sum = 0
  for row1,row2 in zip(mat1,mat2):
    for elem1,elem2 in zip(row1,row2):
      if (elem1 == elem2) & (elem2 == 1):
        sum += 1
  return sum

def find_fitness2(fitness1, fitness2, sum):
  return fitness1 +fitness2-4*sum


size = 9
population = 5000
numberOfDrone=2
selected = int(population/2)
generation = 200
length = ((size*size-1)/numberOfDrone)
mutation_ratio = 0.01
end = copy.deepcopy(start)
routes, fitness_list ,matrix_list = create_first_route(start,numberOfDrone,length)

unzipped_list=group(routes,numberOfDrone,fitness_list,matrix_list)


for i in range (generation-1):
    for j in range(numberOfDrone) :
        cross_generation = cross_over(list(unzipped_list[j])[:2500], length)
        mutated_generation = mutation(cross_generation, mutation_ratio*population, length, population) 
        
        new_routes, new_fitness, new_matrix_list = try_routes(mutated_generation, start, size)
    unzipped_list=group(new_routes,numberOfDrone,new_fitness,new_matrix_list)
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198


In [39]:
print(str(unzipped_list[0][1]))
print(len(unzipped_list[0][0]))

[1, 3, 7, 3, 5, 3, 7, 1, 8, 7, 1, 7, 2, 4, 5, 4, 2, 3, 7, 6, 1, 3, 1, 5, 8, 5, 2, 5, 7, 1, 5, 1, 1, 3, 7, 3, 7, 3, 7, 3, 5]
41


In [42]:
end[0]=start[0]
end[1]=start[1]
print("start:",start)
print("end",end)
matrix = np.zeros((size,size))
for i in range (len(unzipped_list[0])-1):
  add_move =moves[unzipped_list[0][0][i+1]-1]
  end = [end[0] + add_move[0], end[1] + add_move[1]]
  print("end:",end)
  matrix[end[0], end[1]] = 1
print(end)
area = len(np.argwhere(matrix == 1))
print(area)


start: [5, 5]
end [5, 5]
end: [6, 5]
end: [5, 6]
end: [6, 6]
end: [7, 7]
end: [6, 8]
end: [6, 7]
end: [5, 7]
end: [6, 6]
end: [7, 6]
end: [7, 5]
end: [7, 6]
end: [6, 6]
end: [5, 6]
end: [6, 6]
end: [5, 5]
end: [5, 4]
end: [6, 3]
end: [5, 3]
end: [4, 2]
end: [3, 3]
end: [2, 3]
end: [3, 3]
end: [4, 3]
end: [3, 3]
end: [2, 2]
end: [2, 3]
end: [3, 2]
end: [2, 2]
end: [3, 2]
end: [4, 3]
end: [3, 2]
end: [4, 3]
end: [3, 4]
end: [3, 3]
end: [2, 3]
end: [3, 4]
end: [2, 4]
end: [2, 3]
end: [2, 4]
end: [1, 3]


IndexError: ignored